# Used Car Marketplace

In [ ]:
# Instalasi library Faker
!pip install Faker
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Library yang akan digunakan
from faker import Faker
from tabulate import tabulate
import random
from datetime import datetime, timedelta
import csv
import pandas as pd
import re

In [ ]:
# Definisikan bahwa data yang digunakan menggunakan format Indonesia
fake = Faker('id_ID')

## Membuat dummy data

### Dummy data yang dibuat adalah tabel-tabel berikut:

* libraries
* users
* publishers
* authors
* categories
* books
* book at library
* loan
* hold



In [ ]:
cars = pd.read_csv("/content/drive/MyDrive/Pacmann/SQL/vehicles.csv")
cars = cars[['id', 'price', 'year', 'manufacturer', 'model']]
cars = cars.dropna()

In [ ]:
cars.isna().sum()

id              0
price           0
year            0
manufacturer    0
model           0
dtype: int64

In [ ]:
def generate_users_data(num=5):
    """
    Generate user data.

    Args:
        num (int, optional): Number of user records to generate. Defaults to 5.

    Returns:
        pandas.DataFrame: DataFrame containing user data with columns:
            - user_id (int): Unique identifier for each user.
            - username (str): Username generated based on first name, last name, and phone number.
            - first_name (str): User's first name.
            - last_name (str): User's last name.
            - email (str): User's email address.
            - password (str): User's password.
            - contact_number (str): User's contact phone number.
    """

    first_name_list = []
    last_name_list = []
    phone_number_list = []
    password_list = []
    username_list = []
    email_list = []
    address_list = [fake.address() for i in range(num)]
    street_name_list = [fake.street() for i in range(num)]
    state_list = [fake.administrative_unit() for i in range(num)]
    city_list = [fake.city_name() for i in range(num)]
    postal_code_list = [fake.postcode() for i in range(num)]
    longitude_list = [float(fake.longitude()) for i in range(num)]
    latitude_list = [float(fake.latitude()) for i in range(num)]
    is_seller_list = [fake.boolean(chance_of_getting_true = 60) for i in range(num)]

    for i in range(num):
      first_name = fake.first_name()
      last_name = fake.last_name()
      phone_number = fake.phone_number()
      password = fake.password()
      username = f"{first_name.lower()}_{last_name.lower()}{phone_number[-2:-1]}"
      email = f"{first_name.lower()}.{last_name.lower()}@usermail.com"

      first_name_list.append(first_name)
      last_name_list.append(last_name)
      phone_number_list.append(phone_number)
      password_list.append(password)
      username_list.append(username)
      email_list.append(email)

    users = {"user_id": [i+1 for i in range(num)],
              "username": username_list,
              "first_name": first_name_list,
              "last_name": last_name_list,
              "email": email_list,
              "password": password_list,
              "contact_number": phone_number_list,
              "address": address_list,
              "street_name": street_name_list,
              "state": state_list,
              "city": city_list,
              "postal_code": postal_code_list,
              "longitude": longitude_list,
              "latitude": latitude_list,
              "is_seller": is_seller_list
             }

    df = pd.DataFrame(users)
    return df

df_users = pd.DataFrame(generate_users_data(num=200))
df_users.head()

,user_id,username,first_name,last_name,email,password,contact_number,address,street_name,state,city,postal_code,longitude,latitude,is_seller
0,1,febi_wijaya1,Febi,Wijaya,febi.wijaya@usermail.com,i1wGPl(Z(e,+62 (032) 087-1712,"Jalan Peta No. 52\nPagaralam, BT 44702",Peta,Kalimantan Selatan,Ambon,08221,14.089706,-45.490060,True
1,2,murti_zulaika9,Murti,Zulaika,murti.zulaika@usermail.com,1L8HROqd*K,+62 (018) 884-2998,"Gang Soekarno Hatta No. 57\nBau-Bau, MA 07788",Veteran,Sulawesi Utara,Tebingtinggi,69033,-87.783024,89.079686,False
2,3,adikara_pudjiastuti3,Adikara,Pudjiastuti,adikara.pudjiastuti@usermail.com,ULP#V^&m$6,+62 (157) 102 0636,"Gang Wonoayu No. 39\nBontang, JT 41887",Suryakencana,DKI Jakarta,Metro,41557,10.044775,68.916479,False
3,4,jasmani_hidayanto8,Jasmani,Hidayanto,jasmani.hidayanto@usermail.com,%X5sI6xk$S,+62-65-385-4883,"Jalan Lembong No. 03\nMeulaboh, KT 13503",Jend. A. Yani,Jambi,Mojokerto,14533,73.906863,-27.537048,False
4,5,bagus_gunawan6,Bagus,Gunawan,bagus.gunawan@usermail.com,@UeKi0CfFB,+62-0436-164-6167,"Jl. Sadang Serang No. 7\nBandung, KU 74567",Moch. Toha,Kepulauan Bangka Belitung,Manado,58163,14.420214,-20.423169,True


In [ ]:
def generate_cars_data(num, body_type, fuel_type, transmission, color, cars):
    """
    Generate author data.

    Args:
        num (int): Number of author records to generate.
        books_metadata (pandas.DataFrame): DataFrame containing book metadata.

    Returns:
        pandas.DataFrame: DataFrame containing author data with columns:
            - author_id (int): Unique identifier for each author.
            - name (str): Author's name.
            - email (str): Author's email address (generated based on name).
            - contact_number (str): Author's contact phone number (generated).

    """

    car = {"car_id": [i+1 for i in range(num)],
               "brand": [cars['manufacturer'].iloc[i+1] for i in range(num)],
               "model": [cars['model'].iloc[i+1] for i in range(num)],
               "body_type": [random.choice(body_type) for i in range(num)],
               "manufacturing_year": [int(cars['year'].iloc[i+1]) for i in range(num)],
               "color": [random.choice(color) for i in range(num)],
               "mileage_km":[fake.pyint(min_value = 1000, step = 500) for i in range(num)] ,
               "machine_CC":[fake.pyint(min_value = 850, step = 10) for i in range(num)],
               "fuel_type":[random.choice(fuel_type) for i in range(num)],
               "transmission": [random.choice(transmission) for i in range(num)]

              }

    df = pd.DataFrame(car)
    return df

transmission = ["Automatic", "Manual"]
body_type = ["SUV", "MPV", "Sport", "Convertible", "Station Wagon", "Pick Up", "Electric", "Off Road", "Hybrid", "LCGC", "Hatchback", "Crossover", "Coupe"]
fuel_type = ["Gasoline", "Diesel", "Electric Batteries"]
color = ["Grey", "Black", "Silver", "White", "Yellow", "Red"]
df_cars = pd.DataFrame(generate_cars_data(300, body_type, fuel_type, transmission, color, cars))
df_cars.head()

,car_id,brand,model,body_type,manufacturing_year,color,mileage_km,machine_CC,fuel_type,transmission
0,1,chevrolet,silverado 1500,Off Road,2010,Yellow,3000,5210,Electric Batteries,Automatic
1,2,chevrolet,silverado 1500 crew,Coupe,2020,Black,6000,7180,Gasoline,Manual
2,3,toyota,tundra double cab sr,Hatchback,2017,White,5000,2490,Gasoline,Manual
3,4,ford,f-150 xlt,Coupe,2013,Red,5500,1140,Gasoline,Automatic
4,5,gmc,sierra 2500 hd extended cab,Station Wagon,2012,Silver,3000,9700,Diesel,Automatic


In [ ]:
def generate_ads_data(num, df_cars, df_users):
    """
    Generate publisher data.

    Args:
        num (int): Number of publisher records to generate.
        books_metadata (pandas.DataFrame): DataFrame containing book metadata.

    Returns:
        pandas.DataFrame: DataFrame containing publisher data with columns:
            - publisher_id (int): Unique identifier for each publisher.
            - name (str): Publisher's name.
            - address (str): Publisher's address.
            - email (str): Publisher's email address (generated based on name).
            - contact_number (str): Publisher's contact phone number (generated).

    """
    ads = {"ads_id": [i+1 for i in range(num)],
               "title": [f"{df_cars['model'].iloc[i]} - {df_cars['manufacturing_year'].iloc[i]}" for i in range(num)],
               "description": [f"Brand: {df_cars['brand'].iloc[i]}, Color: {df_cars['color'].iloc[i]}, Body_type: {df_cars['body_type'].iloc[i]}" for i in range(num)],
               "price": [float(fake.pyint(min_value = 50_000_000, step = 5_000_000, max_value = 800_000_000)) for i in range(num)] ,
               "bid_allowed": [fake.boolean(chance_of_getting_true = 70) for i in range(num)],
               "user_id": [random.choice(df_users['user_id']) for _ in range(num)],
               "car_id": [random.choice(df_cars['car_id']) for _ in range(num)],
              }

    df = pd.DataFrame(ads)
    return df

df_ads = pd.DataFrame(generate_ads_data(70, df_cars, df_users))
df_ads.head()

,ads_id,title,description,price,bid_allowed,user_id,car_id
0,1,silverado 1500 - 2010,"Brand: chevrolet, Color: Yellow, Body_type: Of...",505000000.0,True,165,209
1,2,silverado 1500 crew - 2020,"Brand: chevrolet, Color: Black, Body_type: Coupe",375000000.0,True,61,166
2,3,tundra double cab sr - 2017,"Brand: toyota, Color: White, Body_type: Hatchback",50000000.0,True,6,299
3,4,f-150 xlt - 2013,"Brand: ford, Color: Red, Body_type: Coupe",740000000.0,True,69,154
4,5,sierra 2500 hd extended cab - 2012,"Brand: gmc, Color: Silver, Body_type: Station ...",315000000.0,True,196,138


In [ ]:
def generate_bids_data(num, df_users, df_ads):
    """
    Generate category data.

    Args:
        num (int): Number of category records to generate.
        cat_name (list): List of category names.

    Returns:
        pandas.DataFrame: DataFrame containing category data with columns:
            - category_id (int): Unique identifier for each category.
            - category_name (str): Name of the category.

    """
    bids = {"bid_id": [i+1 for i in range(num)],
               "bid_price": [float(fake.pyint(min_value = 40_000_000, step = 3_000_000, max_value = 800_000_000)) for i in range(num)],
               "bid_date": [fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime.now()) for i in range(num)],
               "bid_status": [random.choice(['Sent', 'Failed']) for _ in range(num)],
               "user_id": [random.choice(df_users['user_id']) for _ in range(num)],
               "ads_id": [random.choice(df_ads['ads_id']) for _ in range(num)],
              }

    df = pd.DataFrame(bids)
    return df

category_name = ["Mystery", "Thriller", "Science Fiction", "Fantasy", "Romance", "Non-Fiction", "Biography"]
df_bids = pd.DataFrame(generate_bids_data(200, df_users, df_ads))
df_bids

,bid_id,bid_price,bid_date,bid_status,user_id,ads_id
0,1,178000000.0,2024-05-09 23:20:08.160395,Sent,16,24
1,2,364000000.0,2024-01-22 22:43:27.694488,Sent,29,14
2,3,451000000.0,2024-03-14 11:58:09.009171,Failed,20,44
3,4,646000000.0,2024-02-02 02:58:58.354447,Sent,61,38
4,5,70000000.0,2024-03-21 01:22:29.718692,Sent,133,13
...,...,...,...,...,...,...
195,196,685000000.0,2024-01-09 14:04:15.905064,Failed,192,1
196,197,775000000.0,2024-01-06 05:02:25.828142,Sent,162,15
197,198,283000000.0,2024-03-02 11:16:57.694387,Sent,194,20
198,199,229000000.0,2024-04-16 05:45:37.204268,Sent,86,31


In [ ]:
# Save to CSV
df_users.to_csv('users.csv', index=False)
df_cars.to_csv('cars.csv', index=False)
df_bids.to_csv('bids.csv', index=False)
df_ads.to_csv('ads.csv', index=False)